In [1]:
import json
from typing import List, Dict, Optional
from dataclasses import dataclass
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import BaseMessage, AIMessage, HumanMessage

GEMMA = "gemma3:1b"
DEEPSEEK = "deepseek-r1:1.5b"

# Personalized AI Study Plan Generator
Agente LLM para generar planes de estudio personalizados. A continuación se presenta:
- `StudentProfile`: Clase que almacena el perfil del estudiante, incluyendo principalmente fortalezas y debilidades
- `StudyPlanner`: Agente que genera planes de estudio estructurados basándose en: 
  - El tema que quiere estudiar el usuario
  - El perfil académico del estudiante 
- `ChatPlanner`: Agente conversacional que maneja la interacción con el usuario y coordina con el agente de planificación

In [ ]:
@dataclass
class StudentProfile:
  """Perfil del estudiante con sus fortalezas, debilidades y nivel de aprendizaje

  Args:
    strengths (List[str]): temas que domina bien
    weaknesses (List[str]): temas que necesita reforzar
    difficulty_level (str): básico, intermedio, avanzado
  """
  strengths: List[str]   
  weaknesses: List[str] 
  difficulty_level: str 
  
  def to_dict(self) -> Dict:
    return {
      "strengths": self.strengths,
      "weaknesses": self.weaknesses,
      "difficulty_level": self.difficulty_level,
    }

In [3]:
planning_prompt = """ 
Eres un asistente experto en planificación educativa y diseño curricular.
Tu tarea es crear planes de estudio personalizados basándose en:
1. El tema que el estudiante quiere aprender
2. El perfil académico del estudiante (fortalezas y debilidades)

INSTRUCCIONES:
- Analiza las debilidades del estudiante y priorízalas como prerequisitos
- Estructura el plan de manera jerárquica
- Incluye estimaciones de tiempo para cada sección
- Adapta la complejidad al nivel del estudiante
- Mantén coherencia pedagógica (de lo simple a lo complejo)

FORMATO DE RESPUESTA OBLIGATORIO:
Debes responder ÚNICAMENTE con un JSON válido en el siguiente formato exacto:
{{
  "1": {{
    "titulo": "Nombre del tema principal",
    "subtemas": [
      "Subtema 1.1",
      "Subtema 1.2",
      "Subtema 1.3"
    ]
  }},
  "2": {{
    "titulo": "Segundo tema principal", 
    "subtemas": [
      "Subtema 2.1",
      "Subtema 2.2"
    ]
  }}
}}

IMPORTANTE: 
- Responde SOLO con el JSON
- Asegúrate de que el JSON sea válido

Lenguaje de respuesta: Español
"""
# - No incluyas explicaciones ni comentarios fuera del JSON

In [4]:
class StudyPlanner:
  def __init__(self, model_name:str=GEMMA, temperature:float=0.8):
    """

    Args:
        model_name (str, optional): _description_. Defaults to GEMMA.
        temperature (float, optional): _description_. Defaults to 0.8.
    """
    self.model_name = model_name
    self.temperature = temperature
    
    self.planning_prompt = planning_prompt
    self.chat_prompt = ChatPromptTemplate.from_messages(
      [
        ('system', self.planning_prompt),
        ('human', '''
        SOLICITUD DEL ESTUDIANTE: {user_request}
        
        PERFIL DEL ESTUDIANTE:
        - Fortalezas: {strengths}
        - Debilidades: {weaknesses}
        - Nivel de dificultad: {difficulty_level}
        
        Por favor, genera un plan de estudio personalizado.
        ''')
      ]
    )
    self.llm = OllamaLLM(model=model_name, temperature=temperature)
    self.planner = self.chat_prompt | self.llm 
  
  def generate_study_plan(self, user_request:str, student_profile:StudentProfile, attempts:int=4) -> Dict:
    """Genera un plan de estudio personalizado

    Args:
        user_request (str): lo que el estudiante quiere aprender
        student_profile (StudentProfile): perfil académico del estudiante

    Returns:
        Dict: plan de estudio estructurado como diccionario JSON
    """
    try:
      profile_dict = student_profile.to_dict()
      
      response = self.planner.invoke({
        'user_request': user_request,
        'strengths': ', '.join(profile_dict['strengths']),
        'weaknesses': ', '.join(profile_dict['weaknesses']),
        'difficulty_level': profile_dict['difficulty_level'],
      })
      
      # Limpiar la respuesta y parsear JSON
      cleaned_response = self._clean_json_response(response)
      study_plan = json.loads(cleaned_response)
      
      return study_plan
      
    except json.JSONDecodeError as e:
      if attempts != 0: self.generate_study_plan(user_request, student_profile, attempts-1)
      return {
        "error": "Error al parsear JSON del plan de estudio",
        "raw_response": response[:500] if 'response' in locals() else "No response",
        "json_error": str(e),
        "response": response
      }
    except Exception as e:
      if attempts != 0: self.generate_study_plan(user_request, student_profile, attempts-1)
      return {
        "error": f"Error al generar el plan de estudio: {str(e)}",
        "response": response
      }
  
  def _clean_json_response(self, response:str) -> str:
    """Limpia la respuesta del LLM para extraer solo el JSON válido

    Args:
        response (str): _description_

    Returns:
        str: _description_
    """
    # Buscar el JSON en la respuesta
    json_start = response.find('{')
    json_end = response.rfind('}') + 1
        
    if json_start != -1 and json_end != 0:
      json_str = response[json_start:json_end]
      return json_str

    # Si no se encuentra JSON, intentar limpiar la respuesta completa
    cleaned = response.strip()
    if not cleaned.startswith('{'):
      # Buscar patrones comunes y intentar extraer JSON
      lines = cleaned.split('\n')
      json_lines = []
      in_json = False
      
      for line in lines:
        if '{' in line: 
          in_json = True
        if in_json: 
          json_lines.append(line)
        
        if '}' in line and in_json: break
      
      cleaned = '\n'.join(json_lines)
        
    return cleaned

  def analyze_prerequisites(self, topic:str, student_profile:StudentProfile) -> List[str]:
    """Analiza qué prerequisitos necesita el estudiante para el tema solicitado

    Args:
        topic (str): _description_
        student_profile (StudentProfile): _description_

    Returns:
        List[str]: _description_
    """
    analysis_prompt = f"""
    Analiza qué prerequisitos son necesarios para estudiar: {topic}
        
    Debilidades del estudiante: {', '.join(student_profile.weaknesses)}
        
    Identifica qué debilidades son críticas para este tema y devuelve una lista.
    Responde solo con los prerequisitos, uno por línea.
    """
        
    try:
      response = self.llm.invoke(analysis_prompt)
      # Procesar la respuesta para extraer prerequisitos
      prerequisites = [line.strip() for line in response.split('\n') if line.strip()]
      return prerequisites
    except:
      return []

In [5]:
conversation_prompt = """ 
Eres un asistente educativo amigable que ayuda a crear planes de estudio personalizados.

Puedes:
1. Recopilar información sobre el perfil académico del estudiante
2. Entender qué quiere estudiar el usuario
3. Generar planes de estudio personalizados
4. Responder preguntas sobre los planes generados

Sé conversacional, empático y educativo en tus respuestas.
Lenguaje de respuesta: Español
"""

class ChatPlanner:
  "Bot conversacional que usa el agente de planificación"
  def __init__(self, model_name:str=GEMMA, temperature:float=0.8):
    self.conversation_prompt = conversation_prompt
    self.chat_prompt = ChatPromptTemplate.from_messages(
      [
        ('system', self.conversation_prompt),
        MessagesPlaceholder(variable_name='memory'),
        ('human', '{input}')
      ]
    )
    self.llm = OllamaLLM(model=model_name, temperature=temperature)
    self.memory: List[BaseMessage] = []
    self.chatbot = self.chat_prompt | self.llm
    
    # Agente especializado en planificación
    self.planner_agent = StudyPlanner(model_name, temperature=0.7)
    
    # Estado de la conversación
    self.current_student_profile: Optional[StudentProfile] = None
    self.current_topic: Optional[str] = None

  def __call__(self, query:str, profile:StudentProfile):
    "Procesa la consulta del usuario"
    
    # Detectar si el usuario quiere generar un plan
    if self._is_plan_request(query):
      self.current_student_profile = profile
      return self._handle_plan_request(query)
    
    # Conversación normal
    response = self.chatbot.invoke({
      'input': query,
      'memory': self.memory
    })
    
    # Actualizar memoria
    self.memory.append(HumanMessage(content=query))
    self.memory.append(AIMessage(content=response))
    
    return response

  def _is_plan_request(self, query:str) -> bool:
    "Detecta si la consulta es una solicitud de plan de estudio"
    keywords = ['plan', 'estudiar', 'aprender', 'esquema', 'cronograma', 'programa']
    return any(keyword in query.lower() for keyword in keywords)
  
  def _handle_plan_request(self, query:str) -> str:
    "Maneja solicitudes de planes de estudio"
    if not self.current_student_profile:
      return """Para crear un plan de estudio personalizado, necesito conocer tu perfil académico.
      
      Por favor, proporciona la siguiente información:
      1. Temas que dominas bien (fortalezas)
      2. Temas que necesitas reforzar (debilidades)
      3. Nivel de dificultad deseado
      """
    
    # Generar plan con el agente especializado
    plan_dict = self.planner_agent.generate_study_plan(query, self.current_student_profile)
    
    # Convertir a JSON string formateado para mostrar al usuario
    if isinstance(plan_dict, dict) and "error" not in plan_dict:
      plan_json = json.dumps(plan_dict, indent=2, ensure_ascii=False)
      response = f"Plan de estudio generado:\n\n```json\n{plan_json}\n```"
    else:
      # Si hay error, mostrar el diccionario de error
      error_json = json.dumps(plan_dict, indent=2, ensure_ascii=False)
      response = f"Error al generar el plan:\n\n```json\n{error_json}\n```"
    
    # Actualizar memoria
    self.memory.append(HumanMessage(content=query))
    self.memory.append(AIMessage(content=response))
    
    return response
  
  def get_last_plan(self) -> Optional[Dict]:
    "Retorna el último plan generado como diccionario"
    if not self.current_student_profile:
      return None
    # buscar en la memoria el último plan generado
    for message in reversed(self.memory):
      if isinstance(message, AIMessage) and "Plan de estudio generado" in message.content:
        # extraer el JSON del mensaje
        try:
          start = message.content.find('{')
          end = message.content.rfind('}') + 1
          json_str = message.content[start:end]
          return json.loads(json_str)
        except:
          continue
    return None

In [6]:
profile = StudentProfile(
  strengths=["programación básica", "lógica", "resolución de problemas"],
  weaknesses=["matrices", "álgebra lineal", "estadística"],
  difficulty_level="intermedio"
)
request = "Quiero aprender inteligencia artificial y machine learning"


In [7]:
planner = StudyPlanner()
plan_response = planner.generate_study_plan(request, profile)
print(plan_response)

{'1': {'titulo': 'Introducción a la Inteligencia Artificial y Machine Learning', 'subtemas': ['Conceptos básicos de IA y ML (qué es, aplicaciones, etc.)', 'Tipos de aprendizaje automático (supervisado, no supervisado, por refuerzo)', 'El proceso de aprendizaje automático (data, model, test, deploy)', 'Introducción a Python (para la programación básica)', 'Ejemplos prácticos de IA en el mundo real (reconocimiento de imágenes, procesamiento de lenguaje natural, etc.)']}, '2': {'titulo': 'Fundamentos de Machine Learning (Nivel Intermedio)', 'subtemas': ['Regresión Lineal y Logística: Implementación, interpretación y uso', 'Clasificación con Regresión Logística: Tipos, ventajas y desventajas', 'Clasificación con Algoritmos de K-Nearest Neighbors (KNN)', 'Evaluación de Modelos: Métricas de precisión, recall, F1-score, AUC-ROC', 'Selección de Características: Selección de características relevantes y eliminación de ruido']}, '3': {'titulo': 'Profundizando en el Aprendizaje Supervisado', 'sub